In [3]:
import pandas as pd
import numpy as np
import re

# =========================
# CONFIG
# =========================
IMF_PATH = "C:/Users/pnlna/Downloads/dataset_2026-02-21T07_38_10.629962310Z_DEFAULT_INTEGRATION_IMF.STA_CPI_5.0.0.csv"
YEAR = 2023  # change to whatever year you're aligning to (e.g., 2024)
OUTPUT_PATH = f"market_price_volatility_score_{YEAR}.csv"

# Weights (WGI-style weighted components)
W_LEVEL = 0.6
W_VOL = 0.4

# =========================
# HELPERS
# =========================
def safe_minmax(series: pd.Series) -> pd.Series:
    """Min-max normalize to [0,1] with NaN safety."""
    s = pd.to_numeric(series, errors="coerce")
    mn, mx = np.nanmin(s), np.nanmax(s)
    if np.isfinite(mn) and np.isfinite(mx) and mx > mn:
        return (s - mn) / (mx - mn)
    return pd.Series([np.nan] * len(s), index=s.index)

def to_iso3(country_name: str) -> str:
    """
    Best-effort ISO3 conversion. IMF uses country names; we convert for joining.
    If a name doesn't match, iso3 will be None and you can add a manual map.
    """
    try:
        import pycountry
        if pd.isna(country_name):
            return None
        name = str(country_name).strip()

        hit = pycountry.countries.get(name=name)
        if hit:
            return hit.alpha_3

        # Fuzzy match (handles many IMF naming variants)
        hits = pycountry.countries.search_fuzzy(name)
        return hits[0].alpha_3 if hits else None
    except Exception:
        return None

# =========================
# LOAD
# =========================
df = pd.read_csv(IMF_PATH, low_memory=False)

required = {"COUNTRY", "FREQUENCY", "TYPE_OF_TRANSFORMATION"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Month columns like "2023-M01", "2023-M02", ...
month_cols = [c for c in df.columns if re.match(r"^\d{4}-M\d{2}$", str(c))]
if not month_cols:
    raise ValueError("No monthly columns found like '2023-M01'.")

# Keep monthly frequency rows
df = df[df["FREQUENCY"].astype(str).str.lower().eq("monthly")].copy()

# Keep MoM percent change rows (this is your inflation_rate series)
mom_mask = df["TYPE_OF_TRANSFORMATION"].astype(str).str.contains(
    "period-over-period percent change", case=False, na=False
)
df_mom = df[mom_mask].copy()

if df_mom.empty:
    raise ValueError(
        "Could not find MoM percent change series. "
        "Check TYPE_OF_TRANSFORMATION values in your file."
    )

# =========================
# WIDE -> LONG (country, period, mom_inflation)
# =========================
long = df_mom.melt(
    id_vars=["COUNTRY"],
    value_vars=month_cols,
    var_name="period",
    value_name="mom_inflation"
)

# Filter to the selected year
long["year"] = pd.to_numeric(long["period"].str.slice(0, 4), errors="coerce")
long = long[long["year"] == YEAR].copy()

# Clean values
long["mom_inflation"] = pd.to_numeric(long["mom_inflation"], errors="coerce")
long = long.dropna(subset=["mom_inflation"])

# =========================
# AGGREGATE to COUNTRY-YEAR
# =========================
agg = long.groupby("COUNTRY", as_index=False).agg(
    inflation_level_abs_mom=("mom_inflation", lambda x: float(np.mean(np.abs(x)))),
    inflation_volatility_mom=("mom_inflation", lambda x: float(np.std(x, ddof=1)) if len(x) > 1 else np.nan),
    months_observed=("mom_inflation", "count")
)

# Reliability guardrail: volatility needs enough months
agg.loc[agg["months_observed"] < 6, "inflation_volatility_mom"] = np.nan

# =========================
# NORMALIZE + SCORE (WGI-like)
# =========================
agg["level_norm"] = safe_minmax(agg["inflation_level_abs_mom"])
agg["vol_norm"] = safe_minmax(agg["inflation_volatility_mom"])

# If volatility missing, fall back to level only
agg["market_volatility_score"] = np.where(
    agg["vol_norm"].notna(),
    W_LEVEL * agg["level_norm"] + W_VOL * agg["vol_norm"],
    agg["level_norm"]
)

# Add ISO3 for joining with other datasets
agg["iso3"] = agg["COUNTRY"].apply(to_iso3)

# =========================
# OUTPUT
# =========================
out = agg[[
    "iso3",
    "COUNTRY",
    "months_observed",
    "inflation_level_abs_mom",
    "inflation_volatility_mom",
    "level_norm",
    "vol_norm",
    "market_volatility_score",
]].copy()

out["year"] = YEAR
out = out.sort_values("market_volatility_score", ascending=False)

# keep only country and volatility score, overwrite output CSV and show top rows
result = out[["COUNTRY", "market_volatility_score"]].copy()
result = result.sort_values("market_volatility_score", ascending=False)
result.to_csv(OUTPUT_PATH, index=False)
print(result.head(20))

                                          COUNTRY  market_volatility_score
91                                        Lebanon                 1.000000
175                                      Zimbabwe                 0.636513
5                                       Argentina                 0.510158
146                      South Sudan, Republic of                 0.279985
164                          Türkiye, Republic of                 0.256353
139                                  Sierra Leone                 0.221465
122                                      Pakistan                 0.156590
74                      Iran, Islamic Republic of                 0.151243
51   Ethiopia, The Federal Democratic Republic of                 0.140867
153                                      Suriname                 0.130586
47                        Egypt, Arab Republic of                 0.113160
25                                        Burundi                 0.109185
61                       